In [1]:
import pandas as pd

In [2]:
def read_command(path):
    with open(path) as f:
        lines = f.readlines()

    com = []
    count = 0
    for l in lines:
        if count == 0 and '# Example' in l:
            count += 1
        elif count == 1 and l == '#\n':
            count += 1
        elif count == 2 and '########################' in l:
            count += 1
        elif count ==3 and l == '\n':
            count += 1
        elif count == 4:
            com.append(l.strip())
            count = 0

    return com

def seperate_comma(com_lst):
    raw_lst = [[ele if ele[-1] != ',' else ele[:-1] + ' ' + ',' for ele in command.split()] for command in com_lst]
    com_lst = [' '.join(ele) for ele in raw_lst]
    return com_lst

def read_all(num_files):
    com_lst = []
    for i in range(1,num_files+1):
        path = './Examples/EGPSR Examples_' + str(i)+ '.txt'
        # path = './Examples/EGPSR Examples.txt'
        com_lst.extend(read_command(path))

    com_lst = seperate_comma(com_lst)
    return com_lst

num_files = 10
# num_files = 1

com_lst = read_all(num_files)
print(com_lst[3])

print('len: ',len(com_lst))
print(com_lst[:10])

Could you please find -B_find- Linda -B_find_per- , go -B_follow- after -I_follow- her -B_follow_per- , and tell -B_say- the time -I_say-
len:  100000
['Describe -B_say- the objects in the sink to me -I_say- please', 'Salute -B_greet- the youngest -i- person -B_greet_per- in the living -i- room -B_greet_dest-', 'Set -B_take- a cup -B_take_obj- on the cupboard -B_take_dest- and a knife -B_take_obj- on its -B_take_dest- left -I_take_dest- _179-its_ please', 'Could you please find -B_find- Linda -B_find_per- , go -B_follow- after -I_follow- her -B_follow_per- , and tell -B_say- the time -I_say-', 'Could you pinpoint -B_find- the basket -B_find_obj- in the bedroom -B_find_dest-', 'Serve -B_take- some grape -i- juice -B_take_obj- in a cup -B_take_dest-', 'Locate -B_find- William -B_find_per- , accompany -B_follow- it -B_follow_per- , and look -B_find- for -I_find- Alex -B_find_per- at the dining -i- table -B_find_dest- please', 'Look -B_find- for -I_find- the apple -B_find_obj- , pinpoint -

In [3]:
def find_sub_list(sub_list,this_list):
    start = this_list.index(sub_list[0])
    return (start,start+len(sub_list))

def process_person_pro(com_lst,path):
    lines = [line.strip() for line in open(path) if line.strip() != '' and line.strip() != 'it']
    pairs_lst = [(lines[i].split(' '),lines[i+1]) for i in range(0,len(lines)//2,2)] #[([B-referee, I-referee],referral),([B-referee, I-referee],referral),....]
    # print(pairs_lst)
    person_pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        lst = com.split()
        cur_referee  = pairs_lst[0][0]
        cur_referral = pairs_lst[0][1]

        words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
        str_referee = ' '.join(cur_referee).strip()
        if str_referee in words_lst and cur_referral in words_lst:
            # ------------ store command idx ------------
            person_pro_dict['idx'].append(i)

            # ------------ store original command  ------------
            person_pro_dict['words'].append(' '.join(words_lst))

            # ------------ store referral, the pronoun in word command ------------
            referral_idx = words_lst.index(cur_referral)
            person_pro_dict['referral'].append((referral_idx,cur_referral))

            # ------------ store referee, the word being referred ------------
            referee_idx = find_sub_list(cur_referee,words_lst)
            #print(referee_idx)
            person_pro_dict['referee'].append((referee_idx,str_referee))

            pairs_lst.pop(0)
            if len(pairs_lst) == 0:
                break
    return person_pro_dict

# path = './Examples/pros_test.txt'
path = './Examples/pros.txt'

person_pro_dict = process_person_pro(com_lst,path)
per_df = pd.DataFrame(person_pro_dict)
per_df

,idx,referee,referral,words
0,3,"((4, 5), Linda)","(8, her)","Could you please find Linda , go after her , a..."
1,14,"((3, 4), Robert)","(10, him)","Get acquainted with Robert at the entrance , t..."
2,16,"((1, 2), Linda)","(8, her)","Meet Linda at the sink , then greet her"
3,22,"((1, 2), Linda)","(8, her)","Meet Linda at the entrance , then greet her pl..."
4,23,"((2, 3), William)","(9, him)","Please contact William at the bed , then greet..."
...,...,...,...,...
6423,49638,"((3, 4), Francis)","(9, her)","Could you locate Francis at the entrance , esc..."
6424,49652,"((4, 5), Skyler)","(11, him)",Please get acquainted with Skyler at the bookc...
6425,49669,"((6, 7), Robert)","(10, him)","Navigate to the kitchen , locate Robert , and ..."
6426,49693,"((6, 7), Francis)","(10, him)","Enter to the bedroom , pinpoint Francis , and ..."


In [4]:
#{'_178-it_', '_285-it_', '_226-it_', '_226-it_,', '_95-it_', '_286-it_', '_227-it_', '_179-its_'}
def get_it_s_lab(com_lst):
    pro_tag_set = set()
    for com in com_lst:
        lst = com.split()
        for ele in lst:
            if ele[0] == '_':
                pro_tag_set.add(ele)
    return pro_tag_set



#"referral" : The referring wildcard.
#"referee" : The wildcard being referred.
def store_it_s_coref(com_lst):
    clean_com_lst = com_lst.copy()
    pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    pro_tag_set = get_it_s_lab(com_lst)
    for i,com in enumerate(com_lst):

        # if i == 50:
        #     break

        lst = com.split()

        the_tag = None
        for tag in list(pro_tag_set):
            if tag in lst:
                the_tag = [ele for ele in lst if ele[0] == '_'][0]
                break

        if the_tag != None: # has pronoun in this command
            clean_com_lst[i] = ' '.join([ele for ele in lst if ele[0] != '_'])
            # ------------ store command idx ------------
            pro_dict['idx'].append(i)

            words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
            pro_dict['words'].append(' '.join(words_lst))
            # ------------ store referral, the pronoun in word command ------------
            if 'its' in the_tag:
                pro_loc = words_lst.index('its')
                pro_dict['referral'].append((pro_loc,'its'))
            elif 'it' in the_tag:
                pro_loc = words_lst.index('it')
                pro_dict['referral'].append((pro_loc,'it'))

            # ------------ store referee, the word being referred ------------
            raw_loc = lst.index(the_tag)
            search_lst = lst[:raw_loc][::-1]
            obj_tag_count = 0
            i_count = 0
            rev_referee = []
            # print(search_lst)
            for j,ele in enumerate(search_lst):
                if obj_tag_count == 2:
                    rev_referee.append(ele)
                    if search_lst[j] == '-i-':
                        rev_referee.pop()
                        i_count += 1
                        continue

                    elif i_count ==len(rev_referee): # unfinished storing referee
                        continue

                    else:
                        break

                if '_obj-' in ele:
                    obj_tag_count += 1

            referee_lst = rev_referee[::-1]
            referee_loc = find_sub_list(referee_lst,words_lst)
            str_referee = ' '.join(referee_lst).strip()
            pro_dict['referee'].append((referee_loc,str_referee))

    return pro_dict,clean_com_lst

# get_it_s_lab(com_lst)
pro_dict,clean_com_lst = store_it_s_coref(com_lst)
it_df =pd.DataFrame(pro_dict)
it_df

,idx,referee,referral,words
0,2,"((2, 3), cup)","(10, its)",Set a cup on the cupboard and a knife on its l...
1,8,"((3, 4), orange)","(10, it)","Pick up the orange from the dining table , lea..."
2,20,"((3, 4), food)","(10, it)",Pick up a food from the storage table and put ...
3,32,"((2, 3), bowl)","(10, its)",Place a bowl on the bookcase and a spoon on it...
4,36,"((8, 9), snacks)","(12, it)","Navigate to the dining room , locate the snack..."
...,...,...,...,...
10884,99980,"((9, 10), orange)","(16, it)","Look for Mary in the bedroom , take the orange..."
10885,99981,"((12, 13), orange)","(16, it)","Pick up the apple from the side table , pick u..."
10886,99982,"((2, 3), bowl)","(11, it)",Leave a bowl on the storage table and a knife ...
10887,99986,"((3, 4), orange)","(9, it)","Pick up the orange from the sink , hand it to ..."


In [5]:
def merge_df(com_lst,it_df,per_df):
    it_idx = it_df['idx'].tolist()
    per_idx = per_df['idx'].tolist()
    dic = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        if i not in it_idx and i not in per_idx:
            dic['idx'].append(i)
            dic['referee'].append('')
            dic['referral'].append('')
            words_lst = [ele for ele in com.split() if ele[0] != '_' and ele[0] != '-']
            dic['words'].append(' '.join(words_lst))

    rest_df = pd.DataFrame(dic)
    df = it_df.copy().append(per_df)
    df = df.append(rest_df)
    df = df.sort_values('idx')
    return df

pro_df = merge_df(clean_com_lst,it_df,per_df)
pro_df

/tmp/ipykernel_11124/3520584853.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = it_df.copy().append(per_df)
/tmp/ipykernel_11124/3520584853.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(rest_df)


,idx,referee,referral,words
0,0,,,Describe the objects in the sink to me please
1,1,,,Salute the youngest person in the living room
0,2,"((2, 3), cup)","(10, its)",Set a cup on the cupboard and a knife on its l...
0,3,"((4, 5), Linda)","(8, her)","Could you please find Linda , go after her , a..."
2,4,,,Could you pinpoint the basket in the bedroom
...,...,...,...,...
82678,99995,,,Robot please grasp the paprika from the end ta...
82679,99996,,,Tell me how many elders are wearing jackets in...
82680,99997,,,Describe the objects in the side table to me
82681,99998,,,Tell me how many elders are raising their righ...


In [6]:
def gen_pro_labels(pro_df):
    referee_lst = pro_df['referee'].tolist()
    referral_lst = pro_df['referral'].tolist()
    words_lst = [ele.split() for ele in pro_df['words'].tolist()]
    pro_labels = [['O']*len(lst) for lst in words_lst]
    for sample_idx in range(len(words_lst)):
        if referral_lst[sample_idx] != '':
            referral_idx = referral_lst[sample_idx][0]
            pro_labels[sample_idx][referral_idx] = 'referral'

            referee_start = referee_lst[sample_idx][0][0]
            referee_end = referee_lst[sample_idx][0][1]
            pro_labels[sample_idx][referee_start] = 'B-referee'
            length = referee_end - referee_start
            if length > 1:
                pro_labels[sample_idx][referee_start + 1 : referee_end] = ['I-referee']*(length - 1)

    return pro_labels
pro_labels = gen_pro_labels(pro_df)
print(pro_labels[:10])

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-referee', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'referral', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-referee', 'O', 'O', 'O', 'referral', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-referee', 'O', 'O', 'O', 'O', 'O', 'O', 'referral', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [7]:
import itertools

def improve_labels(com_lst):
    res = []
    for i,com in enumerate(com_lst):

        #-b_obj  -->  -B_obj: to fix a labelling error in the command generator
        com = [ele if 'b_' not in ele else ele.replace('b_','B_') for ele in com.split()]
        com = ['-'+ele if ele == 'B_guide_dest-' else ele for ele in com]


        if '-B_say-' not in com and '-i-' not in com:
            res.append(com)
            continue
        elif '-B_say-' in com:
            if '-I_say-' not in com:
                labelled = com
            else:
                b_idx = com.index('-B_say-') + 1
                e_idx = com.index('-I_say-') -1
                first = com[:b_idx]
                end = com[e_idx:]

                need_label = com[b_idx:e_idx]

                #rem all -i- in the 'say' cases
                need_label = [ele for ele in need_label if ele != '-i-']

                interleave = list(itertools.chain(*zip(need_label, ['-I_say-']*len(need_label))))
                labelled = first + interleave + end

                com = labelled.copy()

        #deal with -i-
        elif '-i-' in com:
            string = ' '.join(com)
            while '-i- -i-' in string:
                string = string.replace('-i- -i-', '-i-')

            while '-i- -' in string:
                string = string.replace('-i- -','-')

            com = string.split()


            idx_lst = []
            for idx,ele in enumerate(com):
                if ele == '-i-':
                    if len(idx_lst) == 0:
                        idx_lst.append([idx])
                    elif idx == idx_lst[-1][-1] + 2:
                        idx_lst[-1].append(idx)
                    else:
                        idx_lst.append([idx])
            # if i == 0:
            #     print(com)
            #     print(idx_lst)

            for lst in idx_lst:
                end_label = com[lst[-1]+2]
                #com[lst[0]] = end_label.replace(',','')
                if 'B' in end_label:
                    com[lst[-1]+2] = com[lst[-1]+2].replace('B','I')#.replace(',','')


                for j,idx in enumerate(lst):
                    if j == 0:
                        continue
                    else:
                        label = com[lst[-1]+2]

                        #remove the comma for the earlier tokens (with i) if has comma
                        # if ',' in label:
                        #     label = label.replace(',','')

                        com[idx] = label

            labelled = com.copy()

        res.append(labelled)
    return res

res = improve_labels(clean_com_lst)
' '.join(res[3])


'Could you please find -B_find- Linda -B_find_per- , go -B_follow- after -I_follow- her -B_follow_per- , and tell -B_say- the -I_say- time -I_say-'

In [8]:
def generate_seq_out(res):
    seq_in = []
    seq_out = []
    for i,com in enumerate(res):
        in_lst = []
        clean_out_lst = []
        for ele in com:
#             print('ele; ',ele)
            if ele[0] != '-':
                in_lst.append(ele)
                clean_out_lst.append('O')
            else:
                clean_out_lst[-1] = ele.replace('-','',2)
                if ',' in ele:
                    clean_out_lst[-1] = clean_out_lst[-1].replace(',','')

        seq_in.append(in_lst)
        seq_out.append(clean_out_lst)

    return seq_in, seq_out

seq_in, seq_out = generate_seq_out(res)
print(seq_in[2])
seq_out[2]

['Set', 'a', 'cup', 'on', 'the', 'cupboard', 'and', 'a', 'knife', 'on', 'its', 'left', 'please']


['B_take',
 'O',
 'B_take_obj',
 'O',
 'O',
 'B_take_dest',
 'O',
 'O',
 'B_take_obj',
 'O',
 'B_take_dest',
 'I_take_dest',
 'O']

In [9]:
# need to process the pro df as well
def rem_say(seq_in,seq_out,pro_labels,remove = True):
    if remove:
        say_idx = [i for i,labels in enumerate(seq_out) if 'B_say' in labels]
        seq_in = [seq for i,seq in enumerate(seq_in) if i not in  say_idx]
        seq_out = [seq for i,seq in enumerate(seq_out) if i not in  say_idx]
        seq_pro = [seq for i,seq in enumerate(pro_labels) if i not in  say_idx]
    return seq_in, seq_out, seq_pro

def rem_dups(com_lst, lab_lst, pro_labels,rem = True):
    com_lst = [' '.join(lst) for lst in com_lst]
    lab_lst = [' '.join(lst) for lst in lab_lst]


    com_lst, lab_lst, seq_pro = rem_say(com_lst,lab_lst,pro_labels, remove = rem)

    df = pd.DataFrame (com_lst, columns = ['command'])
    df['label'] = lab_lst
    df['pronouns'] = seq_pro
    df2 = df.drop_duplicates(subset=["command"], keep='first')
    return df2

df = rem_dups(seq_in,seq_out,pro_labels)
df

,command,label,pronouns
0,Salute the youngest person in the living room,B_greet O i I_greet_per O O i I_greet_dest,"[O, O, O, O, O, O, O, O]"
1,Set a cup on the cupboard and a knife on its l...,B_take O B_take_obj O O B_take_dest O O B_take...,"[O, O, B-referee, O, O, O, O, O, O, O, referra..."
2,Could you pinpoint the basket in the bedroom,O O B_find O B_find_obj O O B_find_dest,"[O, O, O, O, O, O, O, O]"
3,Serve some grape juice in a cup,B_take O i I_take_obj O O B_take_dest,"[O, O, O, O, O, O, O]"
4,"Locate William , accompany it , and look for A...",B_find B_find_per O B_follow B_follow_per O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
65826,"Robot please go to the dishwasher , find Micha...",O O B_go O O B_go_dest O B_find B_find_per O O...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
65830,"Find James , look for Linda in the bedroom , a...",B_find B_find_per O B_find I_find B_find_per O...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
65831,"Pinpoint the orange , get the apple from the b...",B_find O B_find_obj O B_take O B_take_obj O O ...,"[O, O, O, O, O, O, B-referee, O, O, O, O, O, O..."
65832,"Pinpoint Robin , retrieve the paprika from the...",B_find B_find_per O B_take O B_take_obj O O B_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [10]:
def write_unique_intent(df):
    label = df['label'].tolist()
    label_lst = [l.split() for l in label]
    # unique_labels
    intents = [ele[2:]  for lst in label_lst for ele in lst  if ele.count('_')==1] #ele[2:] to remove the B_, I_
    #intents.extend(['PAD', 'O'])
    unique_intent_verb = [*set(intents)]
    unique_intent_B = ['B-' + intent for intent in unique_intent_verb]
    unique_intent_I = ['I-' + intent for intent in unique_intent_verb]

    # Interleave
    unique_intent = unique_intent_B + unique_intent_I
    unique_intent[::2] = unique_intent_B
    unique_intent[1::2] = unique_intent_I

    unique_intent = ['PAD','O'] + unique_intent

    with open('intent_label.txt','w') as f:
        for intent in unique_intent:
            f.write(intent)
            f.write('\n')
    return unique_intent

unique_intent = write_unique_intent(df)
unique_intent

['PAD',
 'O',
 'B-greet',
 'I-greet',
 'B-guide',
 'I-guide',
 'B-follow',
 'I-follow',
 'B-find',
 'I-find',
 'B-take',
 'I-take',
 'B-go',
 'I-go',
 'B-know',
 'I-know']

# Train Dev Test split

In [11]:
from sklearn.model_selection import train_test_split

def split(df):
    df_train, df_test_dev = train_test_split(df, test_size=0.25, random_state=42)
    df_dev, df_test = train_test_split(df_test_dev, test_size=0.8, random_state=42)

    env_dict = {'train' : df_train,
               'dev' : df_dev,
               'test' : df_test}
    for df in env_dict.values():
        df.columns = ['seq_in', 'seq_out', 'seq_pro']
    return env_dict

env_dict = split(df)
for env,df in env_dict.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

train 



,seq_in,seq_out,seq_pro
465,Please offer something to eat to all the child...,O B_take B_take_obj I_take_obj I_take_obj O B_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
44727,"Place the fork in the kitchen , grasp the appl...",B_take O B_take_obj O O B_take_dest O B_take O...,"[O, O, O, O, O, O, O, O, O, B-referee, O, O, O..."
8371,"Could you please pinpoint Francis , grasp the ...",O O O B_find B_find_per O B_take O B_take_obj ...,"[O, O, O, O, O, O, O, O, B-referee, O, O, O, O..."
47003,Follow the person wearing a gray jacket in the...,B_follow O i I_follow_per I_follow_per I_follo...,"[O, O, O, O, O, O, O, O, O, O]"
62991,Place a dish on the sink and a spoon on it,B_take O B_take_obj O O B_take_dest O O B_take...,"[O, O, B-referee, O, O, O, O, O, O, O, referral]"
...,...,...,...
7058,"Please go to the living room , find the fruits...",O B_go O O i I_go_dest O B_find O B_find_obj O...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
13586,"Navigate to the bedroom , look for the orange ...",B_go O O B_go_dest O B_find I_find O B_find_ob...,"[O, O, O, O, O, O, O, O, B-referee, O, O, O, r..."
55744,"Find John , get the apple from the cupboard , ...",B_find B_find_per O B_take O B_take_obj O O B_...,"[O, O, O, O, O, B-referee, O, O, O, O, O, O, r..."
875,"Spot James , grasp the paprika from the desk ,...",B_find B_find_per O B_take O B_take_obj O O B_...,"[O, O, O, O, O, B-referee, O, O, O, O, O, O, r..."


------------------------------------------------------------------
dev 



,seq_in,seq_out,seq_pro
56456,Could you please pinpoint Michael at the entra...,O O O B_find B_find_per O O B_find_dest O B_fi...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
10453,Could you bring me the fruits at the center desk,O O B_take B_take_dest O B_take_obj O O B_take...,"[O, O, O, O, O, O, O, O, O, O]"
14432,Bring me some drinks from the bag on the cupbo...,B_take B_take_dest O B_take_obj O O i I_take_s...,"[O, O, O, O, O, O, O, O, O, O, O]"
9051,Give me the left-most food from the bookcase,B_take B_take_dest O i I_take_obj O O B_take_sour,"[O, O, O, O, O, O, O, O]"
63703,"Pinpoint Jennifer , locate Charlie at the dish...",B_find B_find_per O B_find B_find_per O O B_fi...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
12042,"Locate a cup in the kitchen , spot a orange , ...",B_find O B_find_obj O O B_find_dest O B_find O...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3154,Please leave a dish on the cupboard and a knif...,O B_take O B_take_obj O O B_take_dest O O B_ta...,"[O, O, O, B-referee, O, O, O, O, O, O, O, refe..."
11978,"Locate the grape juice , locate Elizabeth at t...",B_find O i I_find_obj O B_find B_find_per O O ...,"[O, O, O, O, O, O, B-referee, O, O, O, O, O, O..."
18010,"Get to know Skyler at the desk , then greet he...",O O B_know B_find_per O O B_find_dest O O B_gr...,"[O, O, O, B-referee, O, O, O, O, O, O, referra..."


------------------------------------------------------------------
test 



,seq_in,seq_out,seq_pro
32631,"Could you look for Mary , retrieve the orange ...",O O B_find I_find B_find_per O B_take O B_take...,"[O, O, O, O, O, O, O, O, B-referee, O, O, O, O..."
48280,"Take the orange , bring it to Charlie at the b...",B_take O B_take_obj O B_take B_take_obj O B_ta...,"[O, O, B-referee, O, O, referral, O, O, O, O, ..."
41401,Bring the paprika to the woman dressed in a wh...,B_take O B_take_obj O O i I_take_dest I_take_d...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
34553,Could you bring the grape juice to the person ...,O O B_take O i I_take_obj O O i I_take_dest I_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
49822,Could you lead the youngest person in the dini...,O O B_guide O i I_follow_per O O i I_follow_so...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
9317,"Set a tableware on the desk , pinpoint Skyler ...",B_take O B_take_obj O O B_take_dest O B_find B...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
56151,"Could you navigate to the bookcase , look for ...",O O B_go O O B_go_dest O B_find I_find B_find_...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9886,"Pinpoint a knife in the kitchen , pick up the ...",B_find O B_find_obj O O B_find_dest O B_take I...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
15140,Salute the person pointing to the right in the...,B_greet O i I_greet_per I_greet_per I_greet_pe...,"[O, O, O, O, O, O, O, O, O, O]"


------------------------------------------------------------------


In [12]:
# def find_sub_list(sub_list,this_list):
#     start = this_list.index(sub_list[0])
#     return (start,start+len(sub_list))

def find_sub_list(subl, l):
    # s_str = ' '.join(subl)
    # f_str = ' '.join(l)
    # num = f_str.count(s_str)
    # print(num)
    # ind_subl = []
    # for i in range(num):
    #     ind_subl.extend([i for i in range(len(l)) if l[i] in subl])
    ind_subl = [i for i in range(len(l)) if l[i] in subl]
    if len(ind_subl) == 1:
        return [ind_subl[0]]
    else:
        return ind_subl#[ind_subl[0], ind_subl[-1]]

l=['1','2','3','4','1','2']
find_sub_list(['1','2'],l)

# t = ([1,2],[3,4])
# d = []
# d = [d.extend(l) for l in t]
# d
#
# a = [None,None,None,None]
# a[[0,1,2]] = [1,1,1]

[0, 1, 4, 5]

In [13]:
def seq_intent_out(com_lst,lab_lst):
    out_int_lst = []

    #polite = ['Please','Could you', 'Robot please', 'Could you please', 'please']
    polite_unique = ['Please','Could you', 'Robot', 'please']

    #sep = ['and', ',' , ', and']
    sep_unique = ['and', ',' ]
    for idx,(com,lab) in enumerate(zip(com_lst,lab_lst)):
        # com = com.split()
        # lab = lab.split()


        # if idx > 10:
        #     return
        sentence = ' '.join(com)

        polite_dict = {key:find_sub_list(key.split(' '),com) for key in polite_unique if key in sentence}
        polite_idx = []
        for ele in polite_dict.values():
            polite_idx.extend(ele)

        sep_dict = {key:find_sub_list(key.split(' '),com) for key in sep_unique if key in sentence}
        sep_idx = []
        for ele in sep_dict.values():
            sep_idx.extend(ele)

        non_intent_idx = polite_idx + sep_idx
        lst = ['O' if i in non_intent_idx else None for i in range(len(lab))]

        intent_lst = []

        B = True
        for i,ele in enumerate(lst):
            if ele == None and B ==True:
                B = False
                intent_lst.append({})
                instance_dict = intent_lst[-1]
                intent_idx = [i]

                if lab[i].count('_') == 2:
                    intent = lab[i].split('_')[1]
                    instance_dict.update({intent:intent_idx})

            elif ele == None and B == False:
                intent_idx.append(i)
                if lab[i].count('_') == 2:
                    intent = lab[i].split('_')[1]
                    instance_dict.update({intent:intent_idx})
                    # print(i)
                    # print(lab[i])

            elif ele == 'O':
                B = True

        intent_seq = [ele for ele in lst]
        for instance_dict in intent_lst:
            for intent, idx_lst in instance_dict.items():
                begin = idx_lst.pop(0)
                intent_seq[begin] = 'B-' + intent
                for i in idx_lst:
                    intent_seq[i] = 'I-' + intent

        out_int_lst.append(intent_seq)
        #print(intent_seq)

        # if idx == 2:
        #     print('  '.join(com))
        #     print('polite_dict: ',polite_dict)
        #     print('sep_dict: ',sep_dict)
        #     print('lst: ', lst)
        #     print('intent_lst: ',intent_lst)
        #     print(intent_seq)
        #     print('-----------------------------------------------------------')
    return out_int_lst


#
# lab_lst = [ele.split() for ele in env_dict['train']['seq_out'].tolist()]
# com_lst = [ele.split() for ele in env_dict['train']['seq_in'].tolist()]
# seq_intent_out(com_lst,lab_lst)

In [14]:
def write(f_name,data_lst, space = True):
    with open(f_name, 'w') as f:
        for lst in data_lst:
            if space == True:
                com = ' '.join(lst)
            else:
                com = ''.join(lst)
            f.write(com)
            f.write('\n')




def gen_files(env_name, data_df):
    # get seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
    #get seq in
    com_lst = [ele.split() for ele in data_df['seq_in'].tolist()]
    #get pro labels lst
    pro_labels = data_df['seq_pro'].tolist()

#     # get intent out
#     int_lst = []
#     for sample_lab in lab_lst:
#         lst = []
#         for ele in sample_lab:
#             if '_' not in ele:
#                 lst.append('O')
#             else:
#                 start = ele.find('_') + 1
#                 if ele.count('_') == 2:
#                     end = ele.rfind('_')
#                     lst.append(ele[start:end])
# #################### turns out we don't need to label the verb ###################
#                 else:
#                     lst.append('O') # verb -> 'O'
# #                     lst.append(ele[start:])
#
#
#         int_lst.append(lst)

    int_lst = seq_intent_out(com_lst,lab_lst)

    # get label
    label = []
    for lab_seq in int_lst:
        lst = []
        for lab in lab_seq:
            if lab == 'O':
                continue
            elif len(lst) == 0:
                lst.append(lab)
            elif lab not in lst:
                lst.append(lab)
        hash_taged = [ele+'#' if i != len(lst)-1 else ele for i,ele in enumerate(lst)]
        label.append(hash_taged)


    # get seq out
    # 1. remove the intent label out of the seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
    lab_lst = [[ele if ele.count('_') == 2 else 'O' for ele in lst] for lst in lab_lst]
    # 2. remove the intentions names in the slot names
    slot_seq = []
    for ite,one_seq in enumerate(lab_lst):
        lst = []
        for slot in one_seq:

            if slot == 'O':
                lst.append('O')
            else:
                # with IBO
                slot_start = slot.find('_',2)+1
                entity_type = slot[slot_start:]
                lst.append(slot[0]+'-'+entity_type)

                #without IBO
                # slot_start = slot.find('_',2)+1
                # lst.append(slot[slot_start:])

        slot_seq.append(lst)

    df = data_df.copy()
    df['label'] = label
    df['seq_intent'] = int_lst
    df['seq_out'] = slot_seq
    df['seq_pro'] = pro_labels


    # write all files
    write(f'./{env_name}/seq.in',com_lst)
    write(f'./{env_name}/seq.out',slot_seq)
    write(f'./{env_name}/seq_intent.out',int_lst)
    write(f'./{env_name}/seq_pro.out',pro_labels)
    write(f'./{env_name}/label',label, False)

    return df


df_train_all = gen_files('train',env_dict['train'])
df_dev_all = gen_files('dev',env_dict['dev'])
df_test_all = gen_files('test',env_dict['test'])

env_dict_all = {'train' : df_train_all,
               'dev' : df_dev_all,
               'test' : df_test_all}

for env,df in env_dict_all.items():
    print(env,'\n')
    display(df)
    print('------------------------------------------------------------------')

train 



,seq_in,seq_out,seq_pro,label,seq_intent
465,Please offer something to eat to all the child...,"[O, O, B-obj, I-obj, I-obj, O, B-dest, I-dest,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, B-take, I-take, I-take, I-take, I-take, I-..."
44727,"Place the fork in the kitchen , grasp the appl...","[O, O, B-obj, O, O, B-dest, O, O, O, B-obj, O,...","[O, O, O, O, O, O, O, O, O, B-referee, O, O, O...","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
8371,"Could you please pinpoint Francis , grasp the ...","[O, O, O, O, B-per, O, O, O, B-obj, O, O, B-so...","[O, O, O, O, O, O, O, O, B-referee, O, O, O, O...","[B-find#, I-find#, B-take#, I-take]","[O, O, O, B-find, I-find, O, B-take, I-take, I..."
47003,Follow the person wearing a gray jacket in the...,"[O, O, O, I-per, I-per, I-per, I-per, O, O, B-...","[O, O, O, O, O, O, O, O, O, O]","[B-follow#, I-follow]","[B-follow, I-follow, I-follow, I-follow, I-fol..."
62991,Place a dish on the sink and a spoon on it,"[O, O, B-obj, O, O, B-dest, O, O, B-obj, B-des...","[O, O, B-referee, O, O, O, O, O, O, O, referral]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
...,...,...,...,...,...
7058,"Please go to the living room , find the fruits...","[O, O, O, O, O, I-dest, O, O, O, B-obj, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-go#, I-go#, B-find#, I-find]","[O, B-go, I-go, I-go, I-go, I-go, O, B-find, I..."
13586,"Navigate to the bedroom , look for the orange ...","[O, O, O, B-dest, O, O, O, O, B-obj, O, O, O, ...","[O, O, O, O, O, O, O, O, B-referee, O, O, O, r...","[B-go#, I-go#, B-find#, I-find#, B-take#, I-take]","[B-go, I-go, I-go, I-go, O, B-find, I-find, I-..."
55744,"Find John , get the apple from the cupboard , ...","[O, B-per, O, O, O, B-obj, O, O, B-sour, O, O,...","[O, O, O, O, O, B-referee, O, O, O, O, O, O, r...","[B-find#, I-find#, B-take#, I-take]","[B-find, I-find, O, B-take, I-take, I-take, I-..."
875,"Spot James , grasp the paprika from the desk ,...","[O, B-per, O, O, O, B-obj, O, O, B-sour, O, O,...","[O, O, O, O, O, B-referee, O, O, O, O, O, O, r...","[B-find#, I-find#, B-take#, I-take]","[B-find, I-find, O, B-take, I-take, I-take, I-..."


------------------------------------------------------------------
dev 



,seq_in,seq_out,seq_pro,label,seq_intent
56456,Could you please pinpoint Michael at the entra...,"[O, O, O, O, B-per, O, O, B-dest, O, O, B-per,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find#, B-guide#, I-guide]","[O, O, O, B-find, I-find, I-find, I-find, I-fi..."
10453,Could you bring me the fruits at the center desk,"[O, O, O, B-dest, O, B-obj, O, O, B-sour, I-sour]","[O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
14432,Bring me some drinks from the bag on the cupbo...,"[O, B-dest, O, B-obj, O, O, O, I-sour, I-sour,...","[O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
9051,Give me the left-most food from the bookcase,"[O, B-dest, O, O, I-obj, O, O, B-sour]","[O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
63703,"Pinpoint Jennifer , locate Charlie at the dish...","[O, B-per, O, O, B-per, O, O, B-dest, O, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-find#, I-find#, B-guide#, I-guide]","[B-find, I-find, O, B-find, I-find, I-find, I-..."
...,...,...,...,...,...
12042,"Locate a cup in the kitchen , spot a orange , ...","[O, O, B-obj, O, O, B-dest, O, O, O, B-obj, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find]","[B-find, I-find, I-find, I-find, I-find, I-fin..."
3154,Please leave a dish on the cupboard and a knif...,"[O, O, O, B-obj, O, O, B-dest, O, O, B-obj, B-...","[O, O, O, B-referee, O, O, O, O, O, O, O, refe...","[B-take#, I-take]","[O, B-take, I-take, I-take, I-take, I-take, I-..."
11978,"Locate the grape juice , locate Elizabeth at t...","[O, O, O, I-obj, O, O, B-per, O, O, O, I-dest,...","[O, O, O, O, O, O, B-referee, O, O, O, O, O, O...","[B-find#, I-find#, B-guide#, I-guide]","[B-find, I-find, I-find, I-find, O, B-find, I-..."
18010,"Get to know Skyler at the desk , then greet he...","[O, O, O, B-per, O, O, B-dest, O, O, O, B-per, O]","[O, O, O, B-referee, O, O, O, O, O, O, referra...","[B-find#, I-find#, B-greet#, I-greet]","[B-find, I-find, I-find, I-find, I-find, I-fin..."


------------------------------------------------------------------
test 



,seq_in,seq_out,seq_pro,label,seq_intent
32631,"Could you look for Mary , retrieve the orange ...","[O, O, O, O, B-per, O, O, O, B-obj, O, O, B-so...","[O, O, O, O, O, O, O, O, B-referee, O, O, O, O...","[B-find#, I-find#, B-take#, I-take]","[O, O, B-find, I-find, I-find, O, B-take, I-ta..."
48280,"Take the orange , bring it to Charlie at the b...","[O, O, B-obj, O, O, B-obj, O, B-dest, O, O, B-...","[O, O, B-referee, O, O, referral, O, O, O, O, ...","[B-take#, I-take]","[B-take, I-take, I-take, O, B-take, I-take, I-..."
41401,Bring the paprika to the woman dressed in a wh...,"[O, O, B-obj, O, O, O, I-dest, I-dest, I-dest,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
34553,Could you bring the grape juice to the person ...,"[O, O, O, O, O, I-obj, O, O, O, I-dest, I-dest...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
49822,Could you lead the youngest person in the dini...,"[O, O, O, O, O, I-per, O, O, O, I-sour, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-follow#, B-guide#, I-guide]","[O, O, B-follow, B-guide, I-guide, I-guide, I-..."
...,...,...,...,...,...
9317,"Set a tableware on the desk , pinpoint Skyler ...","[O, O, B-obj, O, O, B-dest, O, O, B-per, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-take#, I-take#, B-find#, I-find]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
56151,"Could you navigate to the bookcase , look for ...","[O, O, O, O, O, B-dest, O, O, O, B-per, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-go#, I-go#, B-find#, I-find#, B-guide#, I-g...","[O, O, B-go, I-go, I-go, I-go, O, B-find, I-fi..."
9886,"Pinpoint a knife in the kitchen , pick up the ...","[O, O, B-obj, O, O, B-dest, O, O, O, O, B-obj,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find#, B-take#, I-take]","[B-find, I-find, I-find, I-find, I-find, I-fin..."
15140,Salute the person pointing to the right in the...,"[O, O, O, I-per, I-per, I-per, I-per, O, O, B-...","[O, O, O, O, O, O, O, O, O, O]","[B-greet#, I-greet]","[B-greet, I-greet, I-greet, I-greet, I-greet, ..."


------------------------------------------------------------------


In [15]:
def write_unique_slot(env_dict_all):
    label = []
    for df in env_dict_all.values():
        label.extend(df['seq_out'].tolist())

    label_lst = label.copy()

    slots = [ele  for lst in label_lst for ele in lst if ele != 'O'] #ele[2:] to remove the B_, I_
    unique_slot = ['PAD','O'] + [*set(slots)]

    with open('slot_label.txt','w') as f:
        for slot in unique_slot:
            f.write(slot)
            f.write('\n')
    return unique_slot

unique_slot = write_unique_slot(env_dict_all)
unique_slot

['PAD',
 'O',
 'B-obj',
 'B-dest',
 'I-sour',
 'I-obj',
 'I-dest',
 'B-per',
 'B-sour',
 'I-per']

In [16]:
def show_unique_labels(env):
    # uni_slot = [line.strip() for line in open('slot_label.txt', 'r', encoding='utf-8')]
    # uni_intent = [line.strip() for line in open('intent_label.txt', 'r', encoding='utf-8')]

    slot_path =  f'./{env}/seq.out'
    intent_path = f'./{env}/seq_intent.out'

    slot_seq = [line.strip().split() for line in open(slot_path, 'r', encoding='utf-8')]
    intent_seq = [line.strip().split() for line in open(intent_path, 'r', encoding='utf-8')]

    slot_lst = [label for seq in slot_seq for label in seq]
    intent_lst = [label for seq in intent_seq for label in seq]

    env_uni_slot = [*set(slot_lst)]
    env_uni_intent = [*set(intent_lst)]
    print('env: ',env)
    print('Unique intent: ',env_uni_intent,len(env_uni_intent))
    print('Unique slots:  ',env_uni_slot,len(env_uni_slot),'\n')
    return


for env in env_dict_all.keys():
    show_unique_labels(env)

env:  train
Unique intent:  ['I-go', 'I-greet', 'O', 'B-greet', 'I-follow', 'I-find', 'B-guide', 'I-take', 'B-follow', 'I-guide', 'B-find', 'B-go', 'B-take'] 13
Unique slots:   ['B-obj', 'B-dest', 'I-sour', 'O', 'I-obj', 'I-dest', 'B-per', 'B-sour', 'I-per'] 9 

env:  dev
Unique intent:  ['I-go', 'I-greet', 'O', 'B-greet', 'I-follow', 'I-find', 'B-guide', 'I-take', 'I-guide', 'B-follow', 'B-find', 'B-go', 'B-take'] 13
Unique slots:   ['B-obj', 'B-dest', 'I-sour', 'O', 'I-obj', 'I-dest', 'B-per', 'B-sour', 'I-per'] 9 

env:  test
Unique intent:  ['I-go', 'I-greet', 'O', 'B-greet', 'I-follow', 'I-find', 'B-guide', 'I-take', 'B-follow', 'I-guide', 'B-find', 'B-go', 'B-take'] 13
Unique slots:   ['B-obj', 'B-dest', 'I-sour', 'O', 'I-obj', 'I-dest', 'B-per', 'B-sour', 'I-per'] 9 

